### **Knowledge Distillation by On-the-Fly Native Ensemble**

#### **Abstract**
- 现有的**离线蒸馏方法**依赖于一个强大的预训练教师模型
    - 优点：有利于知识的发现和传递
    - 缺点：two-phase training process
- On-the-fly Native Ensemble (ONE) 学习策略，可用于 one-stage online distillation。其**代价**为丢失了高容量的 teacher model
- ONE 训练一个 **单个的多分支网络** ，共同建立一个 strong teacher，用来指导 student 的训练
- ONE 泛化更好，计算效率更高
- 数据集：CIFAR-10, CIFAR-100, SVHN, ImageNet

------------------------------------
#### **Introduction**
- compact
  - Knowledge distillation (KD) 
  - Parameter Binarization
  - Filter Pruning
- KD 的缺陷：
  - 训练时间长
  - two-phase training process
  - large memory

- 为解决上述问题，**前人**提出了**同时蒸馏**算法
    - **具体来说，就是同时训练一组（通常是两个）学生模型，相互学习。也就是 peer teach**
    - **peer teach 有时甚至比 KD 效果更好，有理由认为 large teacher model 往往更容易过拟合**
- 前人提出的 online distillation 也有一系列的缺陷
    - 每个 peer-student 可能只能提供有限的额外知识，导致并不是最优蒸馏
    - 训练多个 student model 会导致额外的成本
    - It requires asynchronous model updating with a notorious need of carefully ordering the operations of prediction and back-propagation across networks
> 作者认为，**这些缺陷都是因为缺乏一个合适的 teacher role in the online distillation process**

- 作者提出的 ONE 算法，**训练成本低，模型泛化能力还更好**
    - multi-branch network
    - 每个 branch 都是一个 student model
    - 通过 ensemble 来构建一个 strong teacher
    - **每个 branch 都有两个损失**
      - softmax cross-entropy loss for matching the ground-truth label distribution
      - distillation loss for aligning the prediction of the branch with the teacher's prediction
- **在 test 时，只需要使用其中一个 branch**

-----------------------------------------
#### **Related Work**
- **Knowledge Distillation**，已有 online distillation 的方法，也就是上文中提到的 peer teach。解决了一些问题，但是仍然有一些缺陷。虽然之后又有后人补充，但仍然有不足。**也就是缺乏 a strong "teacher" model，which limits the ifficacy of knowledge discovery and transfer. 并且计算成本仍然很高**
- **Multi-branch Arichitecture**，(ResNet 就可以看作是一个双分支架构)
    - **作者在这里提到了 grouped convolution，但没什么用**

-------------------------------------------
#### **Knowledge Distillation by On-the-Fly Native Ensemble**

![ONE](../img/ONE.png)

- $L_{ce}$ 中的 $ce$ 表示 cross-entropy
- $L_{kl}$ 中的 $kl$ 表示 KL divergence
- 共享 low-level layers
- Gate：It is constructed by one **FC** layer followed by **batch normalisation, ReLU activation, and softmax**, and uses the same input features as the branches.
- **Gate 会与 每个 branch 的 logit 相乘，也就是去学习每个 branch 的重要性得分**

> **<table><tr><td bgcolor=#D1EEEE>此处 Gate 也去共享底层，仿佛并不合理</td></tr></table>**

但实验表明需要Gate（On Table 5）

- 作者设定 $T=3$

> **具体损失细节见原文**

![Algorithm 1 Knowledge Distillation by On-the-Fly Native Ensemble](../img/Algorithm_1-Knowledge-Distillation-by-On-the-Fly-Native-Ensemble.png)


---------------------------------
#### **Experiments**
- **Experiments setup.** 
  - We implemented all networks and model training procedures **in Pytorch**. For all datasets, we adopted the same experimental settings as [ 27 ; 21 ] for making fair comparisons. We used the **SGD with Nesterov momentum and set the momentum to 0.9**, following a standard learning rate **schedule that drops from 0.1 to 0.01 at 50% training and to 0.001 at 75%**. For the training budget, CIFAR/SVHN/ImageNet used **300/40/90 epochs** respectively. We adopt a **3-branch ONE (m = 2)** design unless stated otherwise. We separate the last block of each backbone net from the parameter sharing (except on ImageNet we separate the last 2 blocks for giving more learning capacity to branches) without extra structural optimisation (see ResNet-110 for example in Fig 1). Most state-of-the-art nets are in block structure designs.

![Table 1: Evaluation of our ONE method on CIFAR and SVHN.Metric: Error rate(%).](../img/Table-1_Evaluation-of-our-ONE-method-on-CIFAR-and-SVHN.Metric_Error-rate.png)


![Table 2: Evaluation of our ONE method on ImageNet.Metric: Error rate(%).](../img/Table-2_Evaluation-of-our-ONE-method-on-ImageNet.Metric_Error-rate.png)

- ONE 算法对于各个网络都有效
- **所有单独的分支具有相似的性能，表明他们在训练期间通过提出的 ONE 教师模型充分一致并相互交换各自的知识**

**Comparison with Distillation Methods**

![Table 3: Comparison with knowledge distillation methods on CIFAR100.](../img/Table-3_Comparison-with-knowledge-distillation-methods-on-CIFAR-100.png)

**Comparison with Ensemble Methods**

![Table 4: Comparison with ensembling methods on CIFAR100](../img/Table-4_Comparison-with-ensembling-methods-on-CIFAR-100.png)

**Model Component Analysis**
1. **ONE Teacher的有效性**：在没有在线蒸馏的情况下，目标网络的测试误差率提高了3.11%（从24.73%到21.62%）。这个结果表明，ONE教师相对于单独的分支模型在性能上具有优越性，ONE教师在训练数据上拟合更好，并在测试数据上表现更好。同时，由于闭环设计，ONE教师也从蒸馏中受益，将其误差率从21.84%降低到21.03%。通过蒸馏，目标模型有效地接近了ONE教师，这表明了教师知识传递的成功。**有趣的是，即使在没有蒸馏的情况下，ONE仍然实现了比普通算法更好的泛化效果**。这表明我们的多分支设计通过同时学习共享的低层次层和更多样化的高层次表示知识，**带来了一些正向的正则化效果**。

2. **不共享低层次层的影响**：没有共享低层次层不仅增加了训练成本（增加了83%），还导致性能较差（误差率增加了0.83%）。**这可能的原因是缺乏多分支正则化效应**，如图2(a)所示。

3. **使用平均集成而不使用门控的影响**：使用平均集成来汇总分支模型，但不使用门控，导致性能下降了0.64%（从22.26%到21.62%）。这表明在构建ONE教师时，自适应地利用分支的多样性有益于性能的提升。

![Table 5: Model component analysis on CIFAR100](../img/Table-5_Model-component-analysis-on-CIFAR100.png)


![Figure 2: Effect of online distillation. Network: ResNet-110.](../img/Figure-2_Effect-of-online-distillation.Network_ResNet-110.png)

![Table 6: Benefit of adding branches to ONE on CIFAR100. Network: ResNet-32](../img/Table-6_Benefit-of-adding-branches-to-ONE-on-CIFAR100.Network_ResNet-32.png)

**Model Generalization Analysis**

- 为什么 ONE 泛化能力更好？
    - **详见原文，不过证明了一些宽度的作用**

**Variants Analysis on ONE's Branches**

---------------------------
#### **Conclusion**

------------------------------------------------------
"Parameter binarization"和"filter pruning"都是用于压缩神经网络的技术，以减少模型的存储需求和计算复杂度，从而使其更适合在资源受限的环境中运行。以下是对这两种技术的简要解释：

1. **Parameter Binarization（参数二值化）**:
   参数二值化是一种将神经网络的权重参数从浮点数值转换为二进制（通常是+1和-1）的技术。这可以大大减小模型的存储开销，因为每个参数只需要一个比特来表示。这种方法的关键思想是，将权重量化为二进制形式，然后在前向和反向传播过程中使用二进制权重来计算梯度。通常，参数二值化技术需要特殊的训练过程，以适应二进制权重。虽然这会损失一些模型性能，但可以在嵌入式设备和移动设备等资源受限的场景中获得显著的模型压缩。

2. **Filter Pruning（滤波器剪枝）**:
   滤波器剪枝是一种通过删除神经网络中的不必要滤波器（也称为卷积核或特征图），来减小模型的尺寸和计算开销的技术。在卷积神经网络中，每个滤波器负责检测输入数据的某些特征。通过分析模型中每个滤波器的贡献，并删除那些对模型性能贡献较小的滤波器，可以实现网络的剪枝。这通常需要进行模型微调，以确保剪枝后的模型保持一定的性能水平。滤波器剪枝可大幅减小模型的参数数量和计算开销，同时保持相对较高的性能。

这两种技术都是用于减小神经网络模型的大小和复杂度，以适应资源受限的环境，例如移动设备、嵌入式系统或边缘计算设备。它们允许在这些设备上部署更轻量级的模型，同时尽可能地保持模型的性能水平。

| 技术          | 描述                                           | 优点                   | 缺点                   |
| :-----------: | :-------------------------------------------: | :--------------------: | :--------------------: |
| 参数二值化    | 将每个参数表示为二进制值（0 或 1）           | 显著减少参数数量       | 可能导致准确性损失     |
| 滤波修剪      | 从神经网络中去除滤波器                        | 可实现高压缩率，无显著准确性损失 | 计算量大               |
| 知识蒸馏      | 训练一个较小的网络来模仿大型网络的输出       | 可实现高压缩率，无显著准确性损失 | 训练较小的网络可能困难 |


--------------------------------------------
**分组卷积（grouped convolution）是一种神经网络卷积操作，它将输入特征图分成多个组，然后对每个组分别进行卷积**。分组卷积可以显著降低卷积操作的计算量，同时保持网络的性能。

在标准卷积中，卷积核会作用于输入特征图的所有通道。在分组卷积中，卷积核会作用于输入特征图的每组通道。例如，如果输入特征图有 128 个通道，分组数为 8，则每个卷积核只会作用于 16 个通道。

分组卷积的计算量与标准卷积的计算量可以通过以下公式计算：

```
计算量 = 输入通道数 * 输出通道数 * 卷积核大小 * 滑动步长
```

在标准卷积中，计算量为输入通道数和输出通道数的乘积。在分组卷积中，计算量为输入通道数和输出通道数的乘积，除以分组数。

分组卷积可以用于构建多分支网络架构。在多分支网络架构中，每个分支可以使用不同的分组数。例如，一个网络可以有两个分支，一个分支使用 4 个分组，另一个分支使用 8 个分组。这可以帮助网络更好地利用不同分组的优势。

以下是分组卷积的一些优点：

* 可以显著降低卷积操作的计算量，同时保持网络的性能。
* 可以用于构建多分支网络架构。
* 可以提高网络的并行性。

以下是分组卷积的一些缺点：

* 需要额外的参数来表示分组。
* 可能需要调整网络的超参数才能获得最佳性能。

分组卷积是一种有效的神经网络压缩技术。它可以显著降低卷积操作的计算量，同时保持网络的性能。